In [4]:
from SimulationSrc import *
import openai
from dotenv import load_dotenv

load_dotenv('.env')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
results = []
# 共告知day天的数据
for day in range(1, 6):
    # 重复模拟20次
    for idx in range(1, 21):
        sl = NewsboySimulator()
        # data = sl.simulate_constOrder(day)
        data = sl.simulate(day)

        print(f'{day} day {idx} times: {data}')
        DemandLambda = sl.get_DemandLambda()
        OrderLambda = sl.get_OrderLambda()
        # 用三种方式推断最佳订货量
        poissonInfer = PoissonInfer(data)
        poissonresult = poissonInfer.infer()
        
        empricalInfer = EmpiricalInfer(data)
        empricalresult = empricalInfer.infer()

        chatGPTInfer = ChatGPTInfer(data)
        chatGPTresult = chatGPTInfer.infer()

        # 同时保存 data
        results.append([day, idx, data, DemandLambda, OrderLambda, poissonresult, empricalresult, chatGPTresult])
        print('finish', day, idx)
# 保存结果
import csv
with open('./results/result_0226_var.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['day', 'idx', 'SampleData', 'DemandLambda', 'OrderLambda' ,'poisson', 'empirical', 'chatGPT'])
    for row in results:
        writer.writerow(row)



1 day 1 times: [(4, 4)]
0.1410056572794157
6.541653640331955
0.07993113114063226
7.591017781150846
0.05514933755093911
8.2516698687778
0.0418301128192107
8.730633163454735
0.033559278682976694
9.10462967452016
0.027944446629705126
9.410476199925691
0.023893733397640057
9.668630958942785
0.020839300498813888
9.891610970735444
0.018457308886371312
10.08761798531376
0.016549892581220735
10.262310130347418
0.014989497181618058
10.419750756171366
0.013690286036223385
10.5629535826845
0.012592425636061489
10.694213830085605
0.011652963085173348
10.815318667722043
0.010840284488165076
10.927686088629217
0.010130617305818376
11.03245950905957
0.009505749340634409
11.130573984604423
0.008951498419567806
11.222803663714728
0.008456659971793628
11.309796503276276
0.0080122672788941
11.392100131707062
0.007611061309553145
11.47018143119077
0.007247104101008045
11.544441580639376
0.006915492376695044
11.615227763310571
0.006612142388339504
11.682842386823333
0.006333626174237063
11.747550422543917


In [12]:
import pandas as pd
from SimulationSrc import *
df = pd.read_csv('./results/result_0226_var.csv')
# 把ChatGPT列中为'None'的行删除
# df = df[df['chatGPT'] != 'None']
df = df[df['poisson'] != 'nan']
# dropna
df = df.dropna()
# df = df[df['poisson'] !='NAN']
# 遍历每一行
# day,idx,DemandLambda,OrderLambda,poisson,emprical,chatGPT
for index, row in df.iterrows():
    DemandLambda = float(row['DemandLambda'])

    poisson_order = int(row['poisson'])
    emprical_order = int(row['empirical'])
    chatGPT_order = int(row['chatGPT'])

    poisson_cost = get_average_cost(poisson_order, DemandLambda)
    emprical_cost = get_average_cost(emprical_order, DemandLambda)
    chatGPT_cost = get_average_cost(chatGPT_order, DemandLambda)


    # 把三种方法的cost和ChatGPT是否beat了poisson和emprical写入csv
    df.at[index, 'poisson_cost'] = poisson_cost
    df.at[index, 'empirical_cost'] = emprical_cost
    df.at[index, 'ChatGPT_cost'] = chatGPT_cost

    # # 判断ChatGPT是否beat了poisson和emprical
    # if chatGPT_cost < poisson_cost:
    #     df.at[index, 'ChatGPT_beat_poisson'] = 1
    # if chatGPT_cost == poisson_cost:
    #     df.at[index, 'ChatGPT_equal_poisson'] = 1

    # if chatGPT_cost < emprical_cost:
    #     df.at[index, 'ChatGPT_beat_empirical'] = 1
    # if chatGPT_cost == emprical_cost:
    #     df.at[index, 'ChatGPT_equal_empirical'] = 1
# 在最后一行补上三种方法的平均cost
poisson_average_cost = df['poisson_cost'].mean()
emprical_average_cost = df['empirical_cost'].mean()
chatGPT_average_cost = df['ChatGPT_cost'].mean()
# df.loc[len(df)+1] = [None, None, None, None, None, None, None, 'average_cost', poisson_average_cost, emprical_average_cost, chatGPT_average_cost]

output_name = './results/result_0226_with_var_order.csv'
df.to_csv(output_name, index=False)

# 重新读取并在最后一行写入三种方法的平均cost
with open(output_name, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([None, None, None, None, None, None, None, 'average_cost', poisson_average_cost, emprical_average_cost, chatGPT_average_cost])